# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
ed = Website("https://virtu-tech.de")
print(ed.get_contents())
print("Links found on the page:")
ed.links

Webpage Title:
Virtù Tech - Software that masters uncertainty
Webpage Contents:
Virtù Tech
About
Services
Privacy
Team
Let's Talk
We turn uncertainty into a product.
AI-powered automation, modern mobile solutions, and custom digital tools for regulated and growing businesses. Built with Virtù. Based in Germany. GDPR-compliant by design.
📩 Let's Talk
What "Virtù" Means to Us
In The Prince, Machiavelli described Virtù as the force that masters chaos — courage, adaptability, strategy. At Virtù Tech, we bring this principle to software. We don't just build tools. We build solutions that confront uncertainty — and win.
🎯
Clarity from complexity
🔁
Adaptive, evolving systems
🛡️
Transparent and ethical tech
Our Expertise
✅
Data Intelligence & Automation
Identify patterns and trends in your data to support better decisions
Detect errors and anomalies in financial or operational workflows
Extract key information from documents and emails
Combine human expertise with smart AI tools to streamline 

['/',
 '#about',
 '#services',
 '#privacy',
 '#profile',
 '#contact',
 '#contact',
 'https://www.linkedin.com/in/vasiliy-klyosov-094b331ba/',
 'mailto:vasiliy@virtu-tech.de']

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://virtu-tech.de - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
#about
#services
#privacy
#profile
#contact
#contact
https://www.linkedin.com/in/vasiliy-klyosov-094b331ba/
mailto:vasiliy@virtu-tech.de


In [11]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/deepseek-ai/DeepSeek-R1-0528',
 '/ResembleAI/chatterbox',
 '/deepseek-ai/DeepSeek-R1-0528-Qwen3-8B',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/osmosis-ai/Osmosis-Structure-0.6B',
 '/models',
 '/spaces/ResembleAI/Chatterbox',
 '/spaces/enzostvs/deepsite',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/wushuang98/Direct3D-S2-v1.0-demo',
 '/spaces/alexnasa/Chain-of-Zoom',
 '/spaces',
 '/datasets/yandex/yambda',
 '/datasets/open-r1/Mixture-of-Thoughts',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/MiniMaxAI/SynLogic',
 '/datasets/Hcompany/WebClick',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs

In [13]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'docs page', 'url': 'https://huggingface.co/docs'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
print(get_all_details("https://virtu-tech.de"))

Found links: {'links': [{'type': 'about page', 'url': 'https://virtu-tech.de#about'}, {'type': 'services page', 'url': 'https://virtu-tech.de#services'}, {'type': 'profile page', 'url': 'https://virtu-tech.de#profile'}, {'type': 'contact page', 'url': 'https://virtu-tech.de#contact'}, {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/vasiliy-klyosov-094b331ba/'}]}
Landing page:
Webpage Title:
Virtù Tech - Software that masters uncertainty
Webpage Contents:
Virtù Tech
About
Services
Privacy
Team
Let's Talk
We turn uncertainty into a product.
AI-powered automation, modern mobile solutions, and custom digital tools for regulated and growing businesses. Built with Virtù. Based in Germany. GDPR-compliant by design.
📩 Let's Talk
What "Virtù" Means to Us
In The Prince, Machiavelli described Virtù as the force that masters chaos — courage, adaptability, strategy. At Virtù Tech, we bring this principle to software. We don't just build tools. We build solutions that confront uncertainty —

In [17]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("Virtu Tech", "https://virtu-tech.de")

Found links: {'links': [{'type': 'about page', 'url': 'https://virtu-tech.de#about'}, {'type': 'services page', 'url': 'https://virtu-tech.de#services'}, {'type': 'profile page', 'url': 'https://virtu-tech.de#profile'}, {'type': 'contact page', 'url': 'https://virtu-tech.de#contact'}, {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/vasiliy-klyosov-094b331ba/'}]}


'You are looking at a company called: Virtu Tech\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nVirtù Tech - Software that masters uncertainty\nWebpage Contents:\nVirtù Tech\nAbout\nServices\nPrivacy\nTeam\nLet\'s Talk\nWe turn uncertainty into a product.\nAI-powered automation, modern mobile solutions, and custom digital tools for regulated and growing businesses. Built with Virtù. Based in Germany. GDPR-compliant by design.\n📩 Let\'s Talk\nWhat "Virtù" Means to Us\nIn The Prince, Machiavelli described Virtù as the force that masters chaos — courage, adaptability, strategy. At Virtù Tech, we bring this principle to software. We don\'t just build tools. We build solutions that confront uncertainty — and win.\n🎯\nClarity from complexity\n🔁\nAdaptive, evolving systems\n🛡️\nTransparent and ethical tech\nOur Expertise\n✅\nData Intelligence & Automation\nIdentify 

In [20]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [21]:
create_brochure("Virtu Tech", "https://virtu-tech.de")

Found links: {'links': [{'type': 'about page', 'url': 'https://virtu-tech.de#about'}, {'type': 'services page', 'url': 'https://virtu-tech.de#services'}, {'type': 'profile page', 'url': 'https://virtu-tech.de#profile'}, {'type': 'contact page', 'url': 'https://virtu-tech.de#contact'}, {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/vasiliy-klyosov-094b331ba/'}]}


# Virtù Tech Brochure

---

## About Virtù Tech

At **Virtù Tech**, we are dedicated to transforming uncertainty into opportunity through the power of technology. Based in Germany, we specialize in AI-powered automation, custom digital tools, and modern mobile solutions designed for regulated and growing businesses. All our products are built with a commitment to GDPR compliance.

### What "Virtù" Means to Us

Inspired by Machiavelli’s principles in *The Prince*, "Virtù" embodies courage, adaptability, and strategy—qualities we apply in our software solutions. We don't merely create tools; we forge solutions that embrace complexity and navigate uncertainty.

---

## Our Services

### Data Intelligence & Automation
- Empower your decision-making with insights drawn from data.
- Detect anomalies in workflows to enhance operational efficiency.
- Extract vital information from documents and communications.

### Custom Software Development
- Transform your ideas into working products with our full-service development.
- From prototypes to long-term support, we ensure clear communication without technical jargon.
- Create tailored mobile and web applications specific to your business needs.

### Business Process Optimization
- Automate manual tasks and digitize workflows to reduce redundancy.
- Integrate critical systems like inventory, CRM, and accounting.
- Develop internal tools that enhance productivity without complexity.

---

## Our Commitment to Security

At Virtù Tech, we prioritize your business's security:
- **GDPR-First by Design**: We ensure 100% compliance with GDPR regulations.
- **Data Residency**: Your data remains securely hosted on German servers—never leaving the EU.
- **ISO 27001 Certified**: We maintain audit-ready and explainable processes to guarantee transparency.

---

## Meet Our Founder

**Vasily Klyosov** - CEO & Founder  
As the driving force behind Virtù Tech, Vasily is passionate about creating meaningful automation through innovative software solutions. With extensive experience in AI, mobile development, and backend architecture, he is committed to mastering uncertainty in the tech landscape.

---

## Company Culture

At Virtù Tech, we foster a collaborative environment that values creativity, adaptability, and continuous learning. Our team thrives on challenges and is dedicated to developing ethical, transparent technology that positively impacts our clients.

---

## Join Our Team

We are always looking for talented individuals who are ready to challenge the status quo and contribute to a culture of innovation. If you’re passionate about technology and making a difference—let’s talk!

---

## Get in Touch

Ready to modernize, automate, or scale your business? We’d love to hear from you!

✉️ **Email**: [vasiliy@virtu-tech.de](mailto:vasiliy@virtu-tech.de)

📩 **Website**: [Virtù Tech](https://www.virtu-tech.de)

---

© 2025 Virtù Tech. All rights reserved.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [22]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [23]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'join page', 'url': 'https://huggingface.co/join'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'forum page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is at the forefront of the AI revolution, bringing together a diverse community dedicated to building innovative machine learning (ML) solutions. Our platform facilitates collaboration among ML enthusiasts and professionals, enabling them to create, share, and deploy sophisticated models, datasets, and applications.

### Our Mission
To empower the AI community by providing state-of-the-art tools and infrastructure that make machine learning accessible and effective for everyone.

## Our Offerings
- **Models**: Access and contribute to a library of 1M+ models designed for a variety of tasks, including natural language processing, image and audio recognition, and more.
- **Datasets**: Browse and utilize over 250k datasets to support your ML projects effectively.
- **Spaces**: Run and deploy applications seamlessly, with a selection of ongoing projects such as Chatterbox TTS and DeepSite.
- **Enterprise Solutions**: Tailored offerings designed for organizations, featuring premium support, advanced security, and dedicated resources.

## Community First
Our vibrant community includes over 50,000 organizations such as Google, Microsoft, Amazon, and many non-profits like AI2. By joining Hugging Face, you become part of a collaborative environment where you can learn and grow alongside industry leaders.

## Our Culture
At Hugging Face, we believe in openness and collaboration. We encourage:
- **Innovation**: Experiment with new technologies and ideas in a supportive environment.
- **Diversity**: We welcome diverse perspectives to tackle challenges in AI and ML.
- **Growth**: We offer opportunities for continuous learning and professional development.

## Careers at Hugging Face
We are always looking for talented individuals to join our team. Our job opportunities span various fields, including software engineering, data science, product management, and community engagement. If you're passionate about AI and want to impact the future of technology, check our [current openings](https://huggingface.co/jobs) to find the right fit for you.

### Join Us!
- **Follow Us on Social Media**: Stay updated on the latest from Hugging Face. Follow us on [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface), and [GitHub](https://github.com/huggingface).
- **Sign Up Today**: Ready to dive into the world of ML? Create your account now and start collaborating: [Sign Up](https://huggingface.co/signup).

## Contact Us
For inquiries or more information about our offerings, don't hesitate to [reach out](mailto:info@huggingface.co).

---

**Hugging Face** – The AI community building the future.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>